In [1]:
import numpy as np
import pandas as pd
import re

import pdpipe as pdp
import matplotlib.pyplot as plt
import glob
import os
import csv
from itertools import islice
from decimal import Decimal
import xml.etree.ElementTree as et
from xml.etree.ElementTree import parse
import openpyxl
import pytest

C:\Users\smvan\anaconda3\lib\site-packages\pdpipe\col_generation.py:12: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [2]:
consolidated_df = pd.read_csv("C:/Users/smvan/CFFS-S23/CFFS-22-23/data/Consolidated_purchases/OK_consol_purchases_march1_april21.csv")

# consolidated_df = pd.read_csv("C:/Users/smvan/CFFS-S23/CFFS-22-23/data/mapping/new items/2023-07-20_New_Items.csv")
consolidated_df.head()

,invoice_counter,invoice_num,invoice_total,item_num,qty,line_amount,adj_line_amount,uom,unit_cost,item_descrip,...,prod_group_descrip,order_code,compute_0022,pak_qty,case_qty,purchase_uom,cat_num,cat_descrip,has_credit,credit_pak_amount
0,252033,M123033004,-172.0,69025,-59.0,-123.9,-123.9,each,2.1,CANDIED CINNAMON ALMOND,...,CANDIES SNACKS,KK051094,N,1.0,1.0,each,7,Candy,N,0
1,252099,0252099,153.3,69025,15.0,31.5,31.5,each,2.1,CANDIED CINNAMON ALMOND,...,CANDIES SNACKS,KK051094,N,1.0,1.0,each,7,Candy,N,0
2,252100,0252099,-153.3,69025,-15.0,-31.5,-31.5,each,2.1,CANDIED CINNAMON ALMOND,...,CANDIES SNACKS,KK051094,N,1.0,1.0,each,7,Candy,N,0
3,252099,0252099,153.3,68933,14.0,29.4,29.4,each,2.1,CANDIED COCONUT ALMO,...,CANDIES SNACKS,KK051093,N,1.0,1.0,each,7,Candy,N,0
4,252100,0252099,-153.3,68933,-14.0,-29.4,-29.4,each,2.1,CANDIED COCONUT ALMO,...,CANDIES SNACKS,KK051093,N,1.0,1.0,each,7,Candy,N,0


In [3]:
# Sort the dataframe by 'item_num' and 'qty'
consolidated_df.sort_values(by=['item_num', 'qty'], inplace=True)

# Find the rows to keep
rows_to_keep = []
prev_item_num = None

for index, row in consolidated_df.iterrows():
    if row['item_num'] != prev_item_num:
        rows_to_keep.append(index)
        prev_item_num = row['item_num']
    elif row['qty'] < 0:
        rows_to_keep.pop()
        rows_to_keep.append(index)

# Filter the dataframe to keep the selected rows
unique_rows = consolidated_df.loc[rows_to_keep]

# Display the unique rows
unique_rows



,invoice_counter,invoice_num,invoice_total,item_num,qty,line_amount,adj_line_amount,uom,unit_cost,item_descrip,...,prod_group_descrip,order_code,compute_0022,pak_qty,case_qty,purchase_uom,cat_num,cat_descrip,has_credit,credit_pak_amount
287,247973,9001094224,8190.64,1016,1.0,69.53,69.53,cs,69.53,BROWNIE ROCKY ROAD 12X16IN TFC,...,BAKED GOODS,5801205,N,1.0,2.0,cs,2,Food,N,0
73,247973,9001094224,8190.64,1020,1.0,57.18,57.18,cs,57.18,BAR NANAIMO SCORED 12X16IN,...,BAKED GOODS,1174120,N,1.0,2.0,cs,2,Food,N,0
215,248086,9001070978,520.60,1028,1.0,42.24,42.24,cs,42.24,BREAD NAAN ORIGINAL TEARDROP,...,BAKED GOODS,1290728,N,1.0,48.0,cs,2,Food,N,0
3381,251703,9001385802,1271.19,1033,1.0,51.20,51.20,cs,51.20,PITA GREEK THK 7IN TFC,...,BREAD,1346507,N,1.0,120.0,cs,2,Food,N,0
3383,247874,9001070944,8370.49,1035,1.0,41.29,41.29,cs,41.29,PITA THK 5IN T/S FZN,...,BREAD,5080747,N,12.0,10.0,cs,2,Food,N,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15225,251066,00022954,641.26,70754,1.0,18.00,18.00,ea,18.00,CK|BS|SPONGE|Seasonal (1sheet),...,PRODUCTION FOOD,70749,N,1.0,1.0,ea,2,Food,N,0
15316,250914,00022966,357.14,70756,1.0,70.00,70.00,ea,70.00,CK|BULK|Seasonal Chia/ Oat 8kg,...,PRODUCTION FOOD,70751,N,1.0,1.0,ea,2,Food,N,0
8603,251629,0000869104,968.35,70806,4.0,192.00,192.00,TUB,48.00,ICE CREAM ESPRESSO 11.4L,...,ICECREAM/NOVELTY/GELATO,IF606112,N,11.4,1.0,TUB,2,Food,N,0
8612,248410,0000860921,1872.55,70837,4.0,192.00,192.00,TUB,48.00,ICE CREAM VANILLA 11.4L,...,ICECREAM/NOVELTY/GELATO,IF606201,N,11.4,1.0,TUB,2,Food,N,0


In [4]:
unique_rows[unique_rows['item_num'] == 69025]

,invoice_counter,invoice_num,invoice_total,item_num,qty,line_amount,adj_line_amount,uom,unit_cost,item_descrip,...,prod_group_descrip,order_code,compute_0022,pak_qty,case_qty,purchase_uom,cat_num,cat_descrip,has_credit,credit_pak_amount
2,252100,0252099,-153.3,69025,-15.0,-31.5,-31.5,each,2.1,CANDIED CINNAMON ALMOND,...,CANDIES SNACKS,KK051094,N,1.0,1.0,each,7,Candy,N,0


In [5]:
categories_df = pd.read_csv("C:/Users/smvan/CFFS-S23/CFFS-22-23/data/GHG_categories/ghge_factors.csv")
categories_df


,Category ID,Food Category,Active Total Supply Chain Emissions (kg CO2 / kg food)
0,1,beef & buffalo meat,41.3463
1,2,lamb/mutton & goat meat,41.6211
2,3,pork (pig meat),9.8315
3,4,"poultry (chicken, turkey)",4.3996
4,5,butter,11.4316
...,...,...,...
58,59,manually adjusted,0.0000
59,60,human labor,0.0000
60,61,kitchen supplies,0.0000
61,62,mushrooms,1.5000


In [6]:
# Convert the "item_descrip" and "Food Category" columns to lowercase
consolidated_df['item_descrip'] = consolidated_df['item_descrip'].str.lower()
categories_df['Food Category'] = categories_df['Food Category'].str.lower()

# Initialize new columns
consolidated_df['Food Category'] = np.nan
consolidated_df['Category ID'] = np.nan

# Define a function to split the Food Category column
# def split_food_category(category):
#     separators = ['/', '(', ',', ')', '|']
#     for separator in separators:
#         category = category.replace(separator, ' ')
#     return category.strip()

def split_food_category(category):
    separators = ['/', '(', ',', ')', '|']
    for separator in separators:
        category = category.replace(separator, ' ')
    return category.strip()

for index, row in categories_df.iterrows():
    categories_df.at[index, 'Food Category'] = split_food_category(row['Food Category'])
    
for index, row in consolidated_df.iterrows():
    consolidated_df.at[index, 'item_descrip'] = split_food_category(row['item_descrip'])
    
    
# Iterate through each row
for index, row in consolidated_df.iterrows():
    item_descrip = row['item_descrip']
    
    # Split the item_descrip into words
    item_words = item_descrip.split()
    
    # Find matches between the words in item_descrip and the Food Category column
    matches = categories_df[categories_df['Food Category'].apply(lambda x: any(word in x.split() for word in item_words))]
    
    # If there is a match, assign the corresponding values
    if not matches.empty:
        consolidated_df.at[index, 'Food Category'] = matches['Food Category'].iloc[0]
        consolidated_df.at[index, 'Category ID'] = matches['Category ID'].iloc[0]

# Print the updated DataFrame
consolidated_df





,invoice_counter,invoice_num,invoice_total,item_num,qty,line_amount,adj_line_amount,uom,unit_cost,item_descrip,...,compute_0022,pak_qty,case_qty,purchase_uom,cat_num,cat_descrip,has_credit,credit_pak_amount,Food Category,Category ID
287,247973,9001094224,8190.64,1016,1.0,69.53,69.53,cs,69.53,brownie rocky road 12x16in tfc,...,N,1.0,2.0,cs,2,Food,N,0,NaN,NaN
289,248381,9001112003,5367.66,1016,1.0,69.53,69.53,cs,69.53,brownie rocky road 12x16in tfc,...,N,1.0,2.0,cs,2,Food,N,0,NaN,NaN
290,248383,9001132129,4406.91,1016,1.0,69.53,69.53,cs,69.53,brownie rocky road 12x16in tfc,...,N,1.0,2.0,cs,2,Food,N,0,NaN,NaN
292,248663,9001144510,9116.11,1016,1.0,69.53,69.53,cs,69.53,brownie rocky road 12x16in tfc,...,N,1.0,2.0,cs,2,Food,N,0,NaN,NaN
293,248997,9001185341,7931.74,1016,1.0,69.53,69.53,cs,69.53,brownie rocky road 12x16in tfc,...,N,1.0,2.0,cs,2,Food,N,0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8752,249360,5200356,213.43,70875,2.0,87.36,87.36,BX,43.68,beef medium grd,...,N,1.0,5.0,BX,2,Food,N,0,beef & buffalo meat,1.0
8754,251688,5204180,265.48,70875,2.0,89.56,89.56,BX,44.78,beef medium grd,...,N,1.0,5.0,BX,2,Food,Y,0,beef & buffalo meat,1.0
8755,252734,5206230,265.48,70875,2.0,89.56,89.56,BX,44.78,beef medium grd,...,N,1.0,5.0,BX,2,Food,Y,0,beef & buffalo meat,1.0
8749,248077,05197159,1042.32,70875,18.0,796.32,796.32,BX,44.24,beef medium grd,...,N,1.0,5.0,BX,2,Food,Y,0,beef & buffalo meat,1.0


In [7]:
pd.DataFrame(consolidated_df.iloc[15644, :])

,15825
invoice_counter,250950
invoice_num,00023085
invoice_total,411.73
item_num,62869
qty,5.0
line_amount,16.2
adj_line_amount,16.2
uom,ea
unit_cost,3.24
item_descrip,ck g&g pot chia pudding.


In [8]:
# for index, row in categories_df.iterrows():
#     for i, r in consolidated_df.iterrows():
#         if consolidated_df.loc[i, 'Food Category'].str.contains(categories_df.loc[index, 'Food Category']):
#             consolidated_df.loc[index, 'Food Category'] = categories_df.loc[index, 'Food Category']
#             consolidated_df.loc[index, 'Category ID'] = categories_df.loc[index, 'Category ID']

In [9]:
# def split_food_category(category):
#     separators = ['/', '(', ',', ')']
#     for separator in separators:
#         category = category.replace(separator, ' ')
#     return category.strip()

# for index, row in categories_df.iterrows():
#     categories_df.at[index, 'Food Category'] = split_food_category(row['Food Category'])

a_df = consolidated_df.iloc[730:760, :]


In [10]:
# Iterate through each row
for index, row in consolidated_df.iterrows():
    item_descrip = row['item_descrip']
    
    # Split the item_descrip into words
    item_words = item_descrip.split()
    
    if 'kale' in item_words or 'lettuce' in item_words or 'romaine' in item_words or 'salad' in item_words:
        consolidated_df.at[index, 'Food Category'] = 'other vegetables'
        consolidated_df.at[index, 'Category ID'] = 40
        
    if 'base' in item_words and 'veg' in item_words or 'veg' in item_words or 'seaweed' in item_words or 'ck|g&g|salad|kalecaesarw/chick' in item_words:
        consolidated_df.at[index, 'Food Category'] = 'other vegetables'
        consolidated_df.at[index, 'Category ID'] = 40
        
    if 'nori' in item_words or 'ck|g&g|salad|kale caesar.' in item_words or 'pickle' in item_words:
        consolidated_df.at[index, 'Food Category'] = 'other vegetables'
        consolidated_df.at[index, 'Category ID'] = 40

        
    if 'spinach' in item_words or 'artichoke' in item_words or 'avocado' in item_words:
        consolidated_df.at[index, 'Food Category'] = 'other vegetables'
        consolidated_df.at[index, 'Category ID'] = 40
        
    if 'cantaloupe' in item_words or 'papaya' in item_words or 'eggplant' in item_words:
        consolidated_df.at[index, 'Food Category'] = 'other fruits'
        consolidated_df.at[index, 'Category ID'] = 31
    
    if 'celery' in item_words or 'coleslaw' in item_words:
        consolidated_df.at[index, 'Food Category'] = 'other vegetables'
        consolidated_df.at[index, 'Category ID'] = 40
    
    if 'sprouts' in item_words:
        consolidated_df.at[index, 'Food Category'] = 'other vegetables'
        consolidated_df.at[index, 'Category ID'] = 40
    
    if 'milkettes' in item_words or ('milk' in item_words and '2%' in item_words) or 'milk' in item_words:
        consolidated_df.at[index, 'Food Category'] = "milk (cow's milk)"
        consolidated_df.at[index, 'Category ID'] = 9
    
    
    if ('milk' in item_words and 'rice' in item_words):
        consolidated_df.at[index, 'Food Category'] = "rice milk"
        consolidated_df.at[index, 'Category ID'] = 29

            
    if 'oat' in item_words or 'oatmeal' in item_words or 'oats' in item_words:
        consolidated_df.at[index, 'Food Category'] = 'oats (oatmeal)'
        consolidated_df.at[index, 'Category ID'] = 23

    if 'dress' in item_words or 'sauce' in item_words or 'jelly' in item_words or 'ketchup' in item_words or 'chutney' in item_words:
        consolidated_df.at[index, 'Food Category'] = 'sauces & paste'
        consolidated_df.at[index, 'Category ID'] = 58

    if 'mayonaise*vegan' in item_words or 'mayonnaise' in item_words or 'mustard' in item_words or 'spread' in item_words or 'paste' in item_words:
        consolidated_df.at[index, 'Food Category'] = 'sauces & paste'
        consolidated_df.at[index, 'Category ID'] = 58
        
        

    # Check if item_descrip contains "cereal"
    if 'cereal' in item_words:
        consolidated_df.at[index, 'Food Category'] = 'other grains/cereals'
        consolidated_df.at[index, 'Category ID'] = 21
              

    if 'coconut' in item_words or 'apricot' in item_words or 'cucumber' in item_words or 'peppers' in item_words or 'salsa' in item_words:
        consolidated_df.at[index, 'Food Category'] = 'other fruits'
        consolidated_df.at[index, 'Category ID'] = 31 

    if 'grapes' in item_words or 'kiwi' in item_words or 'mangoes' in item_words or 'honeydew' in item_words or 'peach' in item_words or 'pears' in item_words:
        consolidated_df.at[index, 'Food Category'] = 'other fruits'
        consolidated_df.at[index, 'Category ID'] = 31 
    
    if 'squash' in item_words or 'watermelon' in item_words or 'zucchini' in item_words:
        consolidated_df.at[index, 'Food Category'] = 'other fruits'
        consolidated_df.at[index, 'Category ID'] = 31 
        
    if 'muffin' in item_words:
        consolidated_df.at[index, 'Food Category'] = 'wheat/rye (bread, pasta, baked goods)'
        consolidated_df.at[index, 'Category ID'] = 24     
  
        
        
    if 'pie' in item_words or 'waffle' in item_words or 'pastry' in item_words or 'flour' in item_words or 'bun' in item_words:
        consolidated_df.at[index, 'Food Category'] = 'wheat/rye (bread, pasta, baked goods)'
        consolidated_df.at[index, 'Category ID'] = 24
    
    if 'bar' in item_words or 'biscuits' in item_words or 'brownie' in item_words or 'cake' in item_words or 'cookie' in item_words:
        consolidated_df.at[index, 'Food Category'] = 'wheat/rye (bread, pasta, baked goods)'
        consolidated_df.at[index, 'Category ID'] = 24
        
    
    if 'loaf' in item_words or 'pretzel' in item_words or 'scone' in item_words or 'tart' in item_words or 'tiramisu' in item_words or 'lady' in item_words:
        consolidated_df.at[index, 'Food Category'] = 'wheat/rye (bread, pasta, baked goods)'
        consolidated_df.at[index, 'Category ID'] = 24

    if 'crouton' in item_words or 'ck|prep|granola' in item_words or 'noodles' in item_words or 'perogy' in item_words or 'pasta' in item_words:
        consolidated_df.at[index, 'Food Category'] = 'wheat/rye (bread, pasta, baked goods)'
        consolidated_df.at[index, 'Category ID'] = 24
    
               
#     if 'co2' in item_words or 'soda' in item_words or 'baking' in item_words:
#         consolidated_df.at[index, 'Food Category'] = 'kitchen supplies'
#         consolidated_df.at[index, 'Category ID'] = 61
            
        
    if 'hashbrown' in item_words or 'potato*pompom' in item_words:
        consolidated_df.at[index, 'Food Category'] = 'potatoes'
        consolidated_df.at[index, 'Category ID'] = 41
    
    if 'dragon' in item_words or 'california' in item_words or 'shrimpmt' in item_words or 'prawn' in item_words or 'shrimp' in item_words or 'prawns' in item_words:
        consolidated_df.at[index, 'Food Category'] = 'crustaceans (shrimp/prawns)'
        consolidated_df.at[index, 'Category ID'] = 13


    if 'cumin' in item_words or 'cloves' in item_words or 'ground' in item_words or 'seasoning' in item_words:
        consolidated_df.at[index, 'Food Category'] = "stimulants & spices misc."
        consolidated_df.at[index, 'Category ID'] = 54
        
    if 'blend' in item_words or 'chili' in item_words or 'pwdr' in item_words or 'caper' in item_words:
        consolidated_df.at[index, 'Food Category'] = "stimulants & spices misc."
        consolidated_df.at[index, 'Category ID'] = 54
    
    if 'leaf' in item_words or 'chile' in item_words or 'paprika' in item_words or 'leaves' in item_words:
        consolidated_df.at[index, 'Food Category'] = "stimulants & spices misc."
        consolidated_df.at[index, 'Category ID'] = 54   
        
    if 'anise' in item_words or 'cardamom' in item_words or 'mix' in item_words or 'wasabi' in item_words:
        consolidated_df.at[index, 'Food Category'] = "stimulants & spices misc."
        consolidated_df.at[index, 'Category ID'] = 54 
        
        
    if 'onion' in item_words or 'leeks' in item_words or 'onions' in item_words or 'shallots' in item_words:
        consolidated_df.at[index, 'Food Category'] = 'onions and leeks'
        consolidated_df.at[index, 'Category ID'] = 39
                
    
    if 'pumpkin' in item_words or 'jackfruit' in item_words or 'pineapple' in item_words or 'raisin' in item_words or 'cherries' in item_words:
        consolidated_df.at[index, 'Food Category'] = 'other fruits'
        consolidated_df.at[index, 'Category ID'] = 31 
        
    if 'pumpkin' in item_words and 'kern' in item_words or 'quinoa' in item_words or 'sesame' in item_words or 'seed' in item_words:
        consolidated_df.at[index, 'Food Category'] = 'tree nuts and seeds'
        consolidated_df.at[index, 'Category ID'] = 26 
        

    if 'tomato' in item_words or 'tomatillos' in item_words or 'tomatoes' in item_words:
        consolidated_df.at[index, 'Food Category'] = 'tomatoes'
        consolidated_df.at[index, 'Category ID'] = 37
        
    if 'soybean' in item_words or 'tofu' in item_words or 'tempeh' in item_words:
        consolidated_df.at[index, 'Food Category'] = 'soybeans/tofu'
        consolidated_df.at[index, 'Category ID'] = 20
    
    if 'sake' in item_words:
        consolidated_df.at[index, 'Food Category'] = 'wine grapes (wine)'
        consolidated_df.at[index, 'Category ID'] = 51
    
    if 'lentils' in item_words or 'ck|g&g|salad|medchickpea' in item_words:
        consolidated_df.at[index, 'Food Category'] = 'other legumes'
        consolidated_df.at[index, 'Category ID'] = 16
    
    if 'chickpea.' in item_words or 'falafel' in item_words or ('patties' in item_words and 'vegan' in item_words):
        consolidated_df.at[index, 'Food Category'] = 'other legumes'
        consolidated_df.at[index, 'Category ID'] = 16
    
    
    if 'tuna' in item_words or 'squid' in item_words or 'cod' in item_words or 'halibut' in item_words or 'salmon' in item_words or 'sal' in item_words:
        consolidated_df.at[index, 'Food Category'] = 'fish (finfish)'
        consolidated_df.at[index, 'Category ID'] = 12
    
    if 'raspberry' in item_words or 'blackberry' in item_words or 'blueberry' in item_words or 'strawberry' in item_words:
        consolidated_df.at[index, 'Food Category'] = 'berries'
        consolidated_df.at[index, 'Category ID'] = 34
        
    if 'raspberries' in item_words or 'blackberries' in item_words or 'blueberries' in item_words or 'strawberries' in item_words:
        consolidated_df.at[index, 'Food Category'] = 'berries'
        consolidated_df.at[index, 'Category ID'] = 34
    
    if 'samosa' in item_words or 'wrap' in item_words or 'sandwich' in item_words or 'soup' in item_words:
        consolidated_df.at[index, 'Food Category'] = 'manually adjusted'
        consolidated_df.at[index, 'Category ID'] = 59
        
    if 'coconut' in item_words and 'milk' in item_words:
        consolidated_df.at[index, 'Food Category'] = 'manually adjusted'
        consolidated_df.at[index, 'Category ID'] = 59
 
    if 'mandarin' in item_words or 'grapefruit' in item_words or 'lemons' in item_words or 'limes' in item_words or 'mandarins' in item_words or 'oranges' in item_words:
        consolidated_df.at[index, 'Food Category'] = 'citrus fruit'
        consolidated_df.at[index, 'Category ID'] = 35
        
    if 'fruit' in item_words or 'fruit.' in item_words:
        consolidated_df.at[index, 'Food Category'] = 'citrus fruit'
        consolidated_df.at[index, 'Category ID'] = 35
    
    if 'olive' in item_words:
        consolidated_df.at[index, 'Food Category'] = 'olives (oil)'
        consolidated_df.at[index, 'Category ID'] = 49
    
    if 'beets' in item_words or 'dill' in item_words or 'ginger' in item_words or 'garlic' in item_words or 'jicama' in item_words:
        consolidated_df.at[index, 'Food Category'] = 'root vegetables'
        consolidated_df.at[index, 'Category ID'] = 38
   
    if 'parsnip' in item_words or 'carrot' in item_words or 'carrots' in item_words or 'turnip' in item_words or 'tapioca' in item_words:
        consolidated_df.at[index, 'Food Category'] = 'root vegetables'
        consolidated_df.at[index, 'Category ID'] = 38
        
        
    if 'thyme' in item_words or 'herb' in item_words:
        consolidated_df.at[index, 'Food Category'] = 'herbs'
        consolidated_df.at[index, 'Category ID'] = 63    
        
    if 'pea' in item_words:
        consolidated_df.at[index, 'Food Category'] = 'peas'
        consolidated_df.at[index, 'Category ID'] = 18 
    
    if 'yogurt' in item_words:
        consolidated_df.at[index, 'Food Category'] = 'yogurt'
        consolidated_df.at[index, 'Category ID'] = 10
        
    
    if 'crepe' in item_words or 'croissant' in item_words or 'danish' in item_words or 'donut' in item_words or 'eclair' in item_words:
        consolidated_df.at[index, 'Food Category'] = 'wheat/rye (bread, pasta, baked goods)'
        consolidated_df.at[index, 'Category ID'] = 24
    
        # Check if prod_group_descrip is "SUPPLY"
#     if row['prod_group_descrip'] == 'SUPPLY':
#         consolidated_df.at[index, 'Food Category'] = 'kitchen supplies'
#         consolidated_df.at[index, 'Category ID'] = 61
        
#     if row['prod_group_descrip'] == 'DISPOSABLES':
#         consolidated_df.at[index, 'Food Category'] = 'kitchen supplies'
#         consolidated_df.at[index, 'Category ID'] = 61

#     if row['prod_group_descrip'] == 'CLEANING':
#         consolidated_df.at[index, 'Food Category'] = 'kitchen supplies'
#         consolidated_df.at[index, 'Category ID'] = 61
        
    if row['prod_group_descrip'] == 'BREAD':
        consolidated_df.at[index, 'Food Category'] = 'wheat/rye (bread, pasta, baked goods)'
        consolidated_df.at[index, 'Category ID'] = 24   
    
    if row['prod_group_descrip'] == 'BAKED GOODS':
        consolidated_df.at[index, 'Food Category'] = 'wheat/rye (bread, pasta, baked goods)'
        consolidated_df.at[index, 'Category ID'] = 24
        
    if row['prod_group_descrip'] == 'CANDIES SNACKS':
        consolidated_df.at[index, 'Food Category'] = 'sugars and sweeteners'
        consolidated_df.at[index, 'Category ID'] = 43
    
    if row['prod_group_descrip'] == 'BEVERAGE' and pd.isna(row['Food Category']):
        consolidated_df.at[index, 'Food Category'] = 'water & beverages'
        consolidated_df.at[index, 'Category ID'] = 55
    
    if 'pringles' in item_words or 'pringle' in item_words or 'potato' in item_words or 'fries' in item_words or 'chip' in item_words:
        consolidated_df.at[index, 'Food Category'] = 'potatoes'
        consolidated_df.at[index, 'Category ID'] = 41
    
    if ('cream' in item_words and 'ice' in item_words) or 'magnum' in item_words or ('mint' in item_words and 'chip' in item_words):
        consolidated_df.at[index, 'Food Category'] = 'ice cream'
        consolidated_df.at[index, 'Category ID'] = 7
    
    if ('base' in item_words and 'chick' in item_words) or ('thigh' in item_words) or ('wings' in item_words) or 'chicken' in item_words or 'turkey' in item_words or 'chick' in item_words:
        consolidated_df.at[index, 'Food Category'] = 'poultry (chicken, turkey)'
        consolidated_df.at[index, 'Category ID'] = 4
    
        # Check if item_descrip contains "proscuitto" or "ham"
    if 'prosciutto' in item_words or 'ham' in item_words or 'bacon' in item_words or 'chorizo' in item_words or 'sausage' in item_words or 'pork' in item_words:
        consolidated_df.at[index, 'Food Category'] = 'pork (pig meat)'
        consolidated_df.at[index, 'Category ID'] = 3

    if 'juice' in item_words or 'pop' in item_words or 'coke' in item_words or 'ale' in item_words or 'inst' in item_words or 'tea' in item_words or 'bev' in item_words or 'punch' in item_words:
        consolidated_df.at[index, 'Food Category'] = 'water & beverages'
        consolidated_df.at[index, 'Category ID'] = 55 
        
    if 'water' in item_words:
        consolidated_df.at[index, 'Food Category'] = 'water & beverages'
        consolidated_df.at[index, 'Category ID'] = 55 
 
    if 'rubbed' in item_words or 'pepper' in item_words or 'powder' in item_words or 'masala' in item_words:
        consolidated_df.at[index, 'Food Category'] = "stimulants & spices misc."
        consolidated_df.at[index, 'Category ID'] = 54
        
    if row['prod_group_descrip'] == 'SUPPLY':
        consolidated_df.at[index, 'Food Category'] = 'kitchen supplies'
        consolidated_df.at[index, 'Category ID'] = 61
        
    if row['prod_group_descrip'] == 'DISPOSABLES':
        consolidated_df.at[index, 'Food Category'] = 'kitchen supplies'
        consolidated_df.at[index, 'Category ID'] = 61

    if row['prod_group_descrip'] == 'CLEANING':
        consolidated_df.at[index, 'Food Category'] = 'kitchen supplies'
        consolidated_df.at[index, 'Category ID'] = 61
        
    
    if 'beer' in item_words:
        consolidated_df.at[index, 'Food Category'] = "barley (beer)"
        consolidated_df.at[index, 'Category ID'] = 50
    
    if 'sugar' in item_words or 'melona' in item_words or 'honey' in item_words or 'syrup' in item_words:
        consolidated_df.at[index, 'Food Category'] = 'sugars and sweeteners'
        consolidated_df.at[index, 'Category ID'] = 43 
        

    if 'salt&vin' in item_words or 'balsamic' in item_words or 'vinegar' in item_words:
        consolidated_df.at[index, 'Food Category'] = 'vinegar'
        consolidated_df.at[index, 'Category ID'] = 57
    
    if 'salt' in item_words:
        consolidated_df.at[index, 'Food Category'] = 'salt'
        consolidated_df.at[index, 'Category ID'] = 56
        
    if 'butter' in item_words:
        consolidated_df.at[index, 'Food Category'] = 'butter'
        consolidated_df.at[index, 'Category ID'] = 5
          
        
    if ('cream' in item_words or 'creamo10%' in item_words) and not 'ice' in item_words:
        consolidated_df.at[index, 'Food Category'] = 'cream'
        consolidated_df.at[index, 'Category ID'] = 8  
        
    if 'paneer' in item_words or 'cheese' in item_words:
        consolidated_df.at[index, 'Food Category'] = 'cheese'
        consolidated_df.at[index, 'Category ID'] = 6
    
    
    if 'egg' in item_words:
        consolidated_df.at[index, 'Food Category'] = 'eggs'
        consolidated_df.at[index, 'Category ID'] = 11
        
    
    if ('milk' in item_words and 'go' in item_words) or ('milk' in item_words and '2%' in item_words) or ('milk' in item_words and 'choc' in item_words):
        consolidated_df.at[index, 'Food Category'] = "milk (cow's milk)"
        consolidated_df.at[index, 'Category ID'] = 9
        
    if 'milk' in item_words and 'homo' in item_words:
        consolidated_df.at[index, 'Food Category'] = "milk (cow's milk)"
        consolidated_df.at[index, 'Category ID'] = 9

    if ('milk' in item_words and 'soy' in item_words):
        consolidated_df.at[index, 'Food Category'] = "soy milk"
        consolidated_df.at[index, 'Category ID'] = 30
        
    if ('milk' in item_words and 'oat' in item_words) or ('oat' in item_words and 'barista' in item_words):
        consolidated_df.at[index, 'Food Category'] = "oat milk"
        consolidated_df.at[index, 'Category ID'] = 28
    
    if 'banana' in item_words:
        consolidated_df.at[index, 'Food Category'] = 'bananas'
        consolidated_df.at[index, 'Category ID'] = 33
    
    if 'mushrooms' in item_words:
        consolidated_df.at[index, 'Food Category'] = 'mushrooms'
        consolidated_df.at[index, 'Category ID'] = 62
        
    if 'corn' in item_words:
        consolidated_df.at[index, 'Food Category'] = 'corn (maize)'
        consolidated_df.at[index, 'Category ID'] = 22
    
    if 'lamb' in item_words or 'mutton' in item_words or 'goat' in item_words:
        consolidated_df.at[index, 'Food Category'] = 'lamb/mutton & goat meat'
        consolidated_df.at[index, 'Category ID'] = 2
        
        
    if 'broccoli' in item_words or 'cabbage' in item_words or 'cauliflower' in item_words or 'kale' in item_words:
        consolidated_df.at[index, 'Food Category'] = 'cabbages and other brassicas (broccoli)'
        consolidated_df.at[index, 'Category ID'] = 36   

    if 'daikon' in item_words or 'brussel' in item_words or 'choy' in item_words or 'broccolini' in item_words or 'gai' in item_words:
        consolidated_df.at[index, 'Food Category'] = 'cabbages and other brassicas (broccoli)'
        consolidated_df.at[index, 'Category ID'] = 36 

    if 'co2' in item_words or 'soda' in item_words or 'baking' in item_words or 'sponge' in item_words:
        consolidated_df.at[index, 'Food Category'] = 'kitchen supplies'
        consolidated_df.at[index, 'Category ID'] = 61
    
    if 'easter' in item_words or 'candy' in item_words or 'sweetener' in item_words or 'puddings' in item_words:
        consolidated_df.at[index, 'Food Category'] = 'sugars and sweeteners'
        consolidated_df.at[index, 'Category ID'] = 43
        
    if 'pumpkin' in item_words or 'hemp' in item_words or 'nut' in item_words or 'almond' in item_words or 'seeds' in item_words or 'chia' in item_words:
        consolidated_df.at[index, 'Food Category'] = 'tree nuts and seeds'
        consolidated_df.at[index, 'Category ID'] = 26
        
    if 'oil' in item_words or 'oils' in item_words:
        consolidated_df.at[index, 'Food Category'] = 'other vegetable oils'
        consolidated_df.at[index, 'Category ID'] = 44

    if ('oil' in item_words and 'soybeans' in item_words) or ('oil' in item_words and 'soybean' in item_words) or ('oil' in item_words and 'soy' in item_words):
        consolidated_df.at[index, 'Food Category'] = 'soybeans (oil)'
        consolidated_df.at[index, 'Category ID'] = 45
    
    if 'oil' in item_words and 'palm' in item_words:
        consolidated_df.at[index, 'Food Category'] = 'palm (oil)'
        consolidated_df.at[index, 'Category ID'] = 46

    if 'oil' in item_words and 'sunflower' in item_words:
        consolidated_df.at[index, 'Food Category'] = 'sunflower (oil)'
        consolidated_df.at[index, 'Category ID'] = 47

    if ('oil' in item_words and 'rapeseed' in item_words) or ('oil' in item_words and 'canola' in item_words):
        consolidated_df.at[index, 'Food Category'] = 'rapeseed/canola (oil)'
        consolidated_df.at[index, 'Category ID'] = 48
        
    if 'bean' in item_words or 'beans' in item_words or 'pulses' in item_words or 'chana' in item_words:
        consolidated_df.at[index, 'Food Category'] = 'beans and pulses (dried)'
        consolidated_df.at[index, 'Category ID'] = 17
    
    
    if 'chocolate' in item_words or 'arnotts' in item_words or 'choco' in item_words or 'cho' in item_words or 'cocoa' in item_words:
        consolidated_df.at[index, 'Food Category'] = 'cocoa'
        consolidated_df.at[index, 'Category ID'] = 52
        
    if 'dough' in item_words or 'cracker' in item_words:
        consolidated_df.at[index, 'Food Category'] = 'wheat/rye (bread, pasta, baked goods)'
        consolidated_df.at[index, 'Category ID'] = 24
    
    if 'rice' in item_words:
        consolidated_df.at[index, 'Food Category'] = 'rice'
        consolidated_df.at[index, 'Category ID'] = 25
        
    if 'apples' in item_words or 'apple' in item_words:
        consolidated_df.at[index, 'Food Category'] = 'apples'
        consolidated_df.at[index, 'Category ID'] = 32  
    
    if ('ice' in item_words and 'cream' in item_words) or 'melona' in item_words:
        consolidated_df.at[index, 'Food Category'] = 'ice cream'
        consolidated_df.at[index, 'Category ID'] = 7
    
        # Check if item_descrip contains "salami"
    if 'salami' in item_words or '50/50' in item_words or 'patties' in item_words or ('beef' in item_words and 'roast' in item_words) or 'beef' in item_words:
        consolidated_df.at[index, 'Food Category'] = 'beef & buffalo meat'
        consolidated_df.at[index, 'Category ID'] = 1
        
    if 'ht' in item_words:
        consolidated_df.at[index, 'Food Category'] = 'ice cream'
        consolidated_df.at[index, 'Category ID'] = 7
    
# consolidated_df
# 18386

#     elif 'chip' in item_words or 'potato' in item_words:
#         consolidated_df.at[index, 'Food Category'] = 'potatoes'
#         consolidated_df.at[index, 'Category ID'] = 41
# Print the updated DataFrame

In [11]:
#a_df = consolidated_df.iloc[421:41, :]
a_df = consolidated_df.iloc[14043:14070, :]
a_df

,invoice_counter,invoice_num,invoice_total,item_num,qty,line_amount,adj_line_amount,uom,unit_cost,item_descrip,...,compute_0022,pak_qty,case_qty,purchase_uom,cat_num,cat_descrip,has_credit,credit_pak_amount,Food Category,Category ID
15157,251005,00023177,317.82,62848,1.0,11.88,11.88,ea,11.88,ck bs scone seasonal 6ea.,...,N,1.0,1.0,ea,2,Food,N,0,"wheat/rye (bread, pasta, baked goods)",24.0
15158,250940,00023189,565.97,62848,1.0,11.88,11.88,ea,11.88,ck bs scone seasonal 6ea.,...,N,1.0,1.0,ea,2,Food,N,0,"wheat/rye (bread, pasta, baked goods)",24.0
15161,250845,00023194,165.05,62848,1.0,11.88,11.88,ea,11.88,ck bs scone seasonal 6ea.,...,N,1.0,1.0,ea,2,Food,N,0,"wheat/rye (bread, pasta, baked goods)",24.0
15162,250869,00023188,184.63,62848,1.0,11.88,11.88,ea,11.88,ck bs scone seasonal 6ea.,...,N,1.0,1.0,ea,2,Food,N,0,"wheat/rye (bread, pasta, baked goods)",24.0
15163,251002,00023191,239.76,62848,1.0,11.88,11.88,ea,11.88,ck bs scone seasonal 6ea.,...,N,1.0,1.0,ea,2,Food,N,0,"wheat/rye (bread, pasta, baked goods)",24.0
15165,251001,00023209,186.79,62848,1.0,11.88,11.88,ea,11.88,ck bs scone seasonal 6ea.,...,N,1.0,1.0,ea,2,Food,N,0,"wheat/rye (bread, pasta, baked goods)",24.0
15166,250936,00023210,565.97,62848,1.0,11.88,11.88,ea,11.88,ck bs scone seasonal 6ea.,...,N,1.0,1.0,ea,2,Food,N,0,"wheat/rye (bread, pasta, baked goods)",24.0
15168,250844,00023224,182.50,62848,1.0,11.88,11.88,ea,11.88,ck bs scone seasonal 6ea.,...,N,1.0,1.0,ea,2,Food,N,0,"wheat/rye (bread, pasta, baked goods)",24.0
15169,250935,00023217,531.95,62848,1.0,11.88,11.88,ea,11.88,ck bs scone seasonal 6ea.,...,N,1.0,1.0,ea,2,Food,N,0,"wheat/rye (bread, pasta, baked goods)",24.0
15170,251000,00023225,149.11,62848,1.0,11.88,11.88,ea,11.88,ck bs scone seasonal 6ea.,...,N,1.0,1.0,ea,2,Food,N,0,"wheat/rye (bread, pasta, baked goods)",24.0


In [12]:
# def split_food_category(category):
#     separators = ['/', '(', ',', ')']
#     for separator in separators:
#         category = category.replace(separator, ' ')
#     return category.strip()

# for index, row in categories_df.iterrows():
#     categories_df.at[index, 'Food Category'] = split_food_category(row['Food Category'])
    
# # Iterate through each row
# for index, row in consolidated_df.iterrows():
#     item_descrip = row['item_descrip']
    
#     # Check if "Food Category" is NaN
#     if pd.isna(row['Food Category']):
    
#         # Split the item_descrip into words
#         item_words = item_descrip.split()
        
#         # Find matches between the words in item_descrip and the Food Category column
#         matches = categories_df[categories_df['Food Category'].apply(lambda x: any(word in x.split() for word in item_words))]
        
#         # If there is a match, assign the corresponding values
#         if not matches.empty:
#             consolidated_df.at[index, 'Food Category'] = matches['Food Category'].iloc[0]
#             consolidated_df.at[index, 'Category ID'] = matches['Category ID'].iloc[0]

# # Print the updated DataFrame
# consolidated_df

In [13]:
filtered_df = consolidated_df[~consolidated_df["Category ID"].isna()]
filtered_df

# filtered_df = a_df[~a_df["Category ID"].isna()]
# filtered_df


,invoice_counter,invoice_num,invoice_total,item_num,qty,line_amount,adj_line_amount,uom,unit_cost,item_descrip,...,compute_0022,pak_qty,case_qty,purchase_uom,cat_num,cat_descrip,has_credit,credit_pak_amount,Food Category,Category ID
287,247973,9001094224,8190.64,1016,1.0,69.53,69.53,cs,69.53,brownie rocky road 12x16in tfc,...,N,1.0,2.0,cs,2,Food,N,0,"wheat/rye (bread, pasta, baked goods)",24.0
289,248381,9001112003,5367.66,1016,1.0,69.53,69.53,cs,69.53,brownie rocky road 12x16in tfc,...,N,1.0,2.0,cs,2,Food,N,0,"wheat/rye (bread, pasta, baked goods)",24.0
290,248383,9001132129,4406.91,1016,1.0,69.53,69.53,cs,69.53,brownie rocky road 12x16in tfc,...,N,1.0,2.0,cs,2,Food,N,0,"wheat/rye (bread, pasta, baked goods)",24.0
292,248663,9001144510,9116.11,1016,1.0,69.53,69.53,cs,69.53,brownie rocky road 12x16in tfc,...,N,1.0,2.0,cs,2,Food,N,0,"wheat/rye (bread, pasta, baked goods)",24.0
293,248997,9001185341,7931.74,1016,1.0,69.53,69.53,cs,69.53,brownie rocky road 12x16in tfc,...,N,1.0,2.0,cs,2,Food,N,0,"wheat/rye (bread, pasta, baked goods)",24.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8752,249360,5200356,213.43,70875,2.0,87.36,87.36,BX,43.68,beef medium grd,...,N,1.0,5.0,BX,2,Food,N,0,beef & buffalo meat,1.0
8754,251688,5204180,265.48,70875,2.0,89.56,89.56,BX,44.78,beef medium grd,...,N,1.0,5.0,BX,2,Food,Y,0,beef & buffalo meat,1.0
8755,252734,5206230,265.48,70875,2.0,89.56,89.56,BX,44.78,beef medium grd,...,N,1.0,5.0,BX,2,Food,Y,0,beef & buffalo meat,1.0
8749,248077,05197159,1042.32,70875,18.0,796.32,796.32,BX,44.24,beef medium grd,...,N,1.0,5.0,BX,2,Food,Y,0,beef & buffalo meat,1.0


In [14]:
consolidated_df[(consolidated_df['Category ID'] == 40.0) & (~consolidated_df['item_num'].duplicated(keep=False))]

,invoice_counter,invoice_num,invoice_total,item_num,qty,line_amount,adj_line_amount,uom,unit_cost,item_descrip,...,compute_0022,pak_qty,case_qty,purchase_uom,cat_num,cat_descrip,has_credit,credit_pak_amount,Food Category,Category ID
10561,249863,1061820707,1651.70,4606,1.0,77.55,77.55,cs,77.55,celery roothl,...,N,1.0,25.0,cs,2,Food,N,0,other vegetables,40.0
6596,250453,1061822848,1301.85,30178,1.0,25.85,25.85,pak,25.85,nori sheets,...,N,1.0,100.0,pak,2,Food,N,0,other vegetables,40.0
6036,251531,9001425820,8341.31,63034,2.0,123.78,123.78,cs,61.89,burger veg malibu gardenburger,...,N,1.0,48.0,cs,2,Food,Y,0,other vegetables,40.0
11948,248479,GP03-06-2023,105.00,69726,-5.0,-15.00,-15.00,HEAD,3.00,lettuce red looseleaf,...,N,1.0,1.0,HEAD,2,Food,N,0,other vegetables,40.0


In [15]:
consolidated_df.to_csv('C:/Users/smvan/CFFS-S23/CFFS-22-23/data/GHG_categories/GHG_assigned_data.csv', index = False)

In [16]:
unfiltered_df = consolidated_df[consolidated_df["Category ID"].isna()]
unfiltered_df

,invoice_counter,invoice_num,invoice_total,item_num,qty,line_amount,adj_line_amount,uom,unit_cost,item_descrip,...,compute_0022,pak_qty,case_qty,purchase_uom,cat_num,cat_descrip,has_credit,credit_pak_amount,Food Category,Category ID
8938,252882,05206281,637.36,2319,20.02,236.64,236.64,Kg,11.82,smokies 5.5 ct bavarian,...,N,1.0,1.0,Kg,2,Food,N,0,NaN,NaN
8937,250210,05201339,2281.04,2319,100.01,1182.12,1182.12,Kg,11.82,smokies 5.5 ct bavarian,...,N,1.0,1.0,Kg,2,Food,N,0,NaN,NaN
6271,248468,9001156743,376.51,3283,3.00,100.62,100.62,500ML,33.54,extract vanilla clear artif,...,N,4.0,1.0,cs,2,Food,N,0,NaN,NaN
6272,250162,9001283697,524.91,3283,3.00,100.62,100.62,500ML,33.54,extract vanilla clear artif,...,N,4.0,1.0,cs,2,Food,N,0,NaN,NaN
6274,248357,9001081626,3645.59,3284,1.00,33.54,33.54,cs,33.54,extract vanilla pure,...,N,1.0,1.0,cs,2,Food,N,0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14552,251891,00023291,479.63,68678,3.00,45.00,45.00,ea,15.00,ck bs filling seasonal 1kg,...,N,1.0,1.0,ea,2,Food,N,0,NaN,NaN
14553,251895,00023315,377.08,68678,3.00,45.00,45.00,ea,15.00,ck bs filling seasonal 1kg,...,N,1.0,1.0,ea,2,Food,N,0,NaN,NaN
14554,251896,00023332,547.24,68678,3.00,45.00,45.00,ea,15.00,ck bs filling seasonal 1kg,...,N,1.0,1.0,ea,2,Food,N,0,NaN,NaN
14555,253396,00023405,429.52,68678,3.00,45.00,45.00,ea,15.00,ck bs filling seasonal 1kg,...,N,1.0,1.0,ea,2,Food,N,0,NaN,NaN


**Unit conversions**

Finding out percentage of items would need manual adjustments to units and don’t come in standard units like kg, lbs, oz, ml, etc.

In [17]:
consolidated_df['uom'].unique()

array(['cs', 'CA', 'CT', '6 pk', 'ea', 'LOAF', 'TUB', 'bottle', '750ML',
       '500ML', 'each', 'BIB 10L', '341ML', '473ML', 'pak', 'JUG', 'pail',
       'Kg', '4L', 'bag', 'un', 'BX', 'box', 'PL', 'LG CAN', 'BG', 'can',
       'jar', 'lb', 'BASKET', 'BUNCH', 'PAK 3', '20UN', 'SHEET', 'L',
       'LAYER', '5KG', '355ML', '10LT', 'DOZ', 'ROLL', 'HEAD'],
      dtype=object)

In [18]:
consolidated_df['case_uom'].unique()

array(['SHEET', 'CT', 'DOZ', 'each', 'CAKE', 'ea', 'g', 'pak', 'LOAF',
       'lb', 'bottle', '750ML', '500ML', 'BIB 20L', '355ML', 'BIB 10L',
       'cs', '591ML', '341ML', 'can', '473ML', '710ml', 'JUG', 'L',
       'LG CAN', 'pail', 'Kg', '4L', 'bag', 'PTN', 'POUCH', 'un', '2L',
       'box', 'PIE', 'BLOCK', 'SM CAN', 'jar', '6L', 'BAR', 'BASKET',
       'HEAD', 'BUNCH', '5#', 'oz', 'PAK 3', 'SLEEVE', '100UN', '20UN',
       '500UN', 'TUB', '340ML', 'fl oz', '4KG', '414ML', '10LT', 'ROLL'],
      dtype=object)

In [19]:
for index, row in consolidated_df.iterrows():
    if row['uom'] == 'Kg':
        consolidated_df.at[index, 'uom'] = 'kg'
    if row['uom'] == 'lbs':
        consolidated_df.at[index, 'uom'] = 'lb'

In [20]:
import numpy as np

# Convert measurements to lowercase
unique_uom = consolidated_df['uom'].str.lower().unique()

# Calculate % of non-standard units
standard_units = ['kg', 'g', 'lb', 'oz', 'ml', 'L']
non_standard_count = np.sum([uom not in standard_units for uom in unique_uom])
percentage_non_standard = (non_standard_count / len(unique_uom)) * 100

print(f"Percentage of items needing manual adjustments to units: {percentage_non_standard:.2f}%")


Percentage of items needing manual adjustments to units: 95.24%


In [21]:
# import re

# consolidated_df['new_uom'] = None
# consolidated_df['total_weight'] = None

# for index, row in consolidated_df.iterrows():
#     item_descrip = re.findall(r'[a-zA-Z]+|\d+', str(row['item_descrip']))
#     consolidated_df.at[index, 'item_descrip'] = item_descrip
    
#     if "ml" in item_descrip:  # Check if "ml" is present in the item description
#         consolidated_df.at[index, 'new_uom'] = 'ml'
    
#     descrip = ' '.join(item_descrip)
#     match = re.search(r'\d+', descrip)  # Search for a numeric value in the description
#     if match:
#         numeric_val = int(match.group(0))
#         consolidated_df.at[index, 'total_weight'] = numeric_val * abs(row['qty'])
        

# import re

# consolidated_df['new_uom'] = None
# consolidated_df['total_weight'] = None

# for index, row in consolidated_df.iterrows():
#     item = re.findall(r'[a-zA-Z]+|\d+', str(row['item_descrip']))
    
#     if "ml" in item:  # Check if "ml" is present in the item description
#         consolidated_df.at[index, 'new_uom'] = 'ml'
    
#     if "lb" in item:  # Check if "ml" is present in the item description
#         consolidated_df.at[index, 'new_uom'] = 'lb'
    
#     descrip = ' '.join(item)
#     match = re.search(r'\d+', descrip)  # Search for a numeric value in the description
#     if match:
#         numeric_val = int(match.group(0))
#         consolidated_df.at[index, 'total_weight'] = numeric_val * abs(row['qty'])


import re

consolidated_df['new_uom'] = None
consolidated_df['total_weight'] = None

for index, row in consolidated_df.iterrows():
    item_descrip = row['item_descrip']
    item = re.findall(r'[a-zA-Z]+[\w%]*', item_descrip)
    
    if "ml" in item:
        consolidated_df.at[index, 'new_uom'] = 'ml'
        
    if "L" in item:
        consolidated_df.at[index, 'new_uom'] = 'L'
    
    if "lb" in item:
        consolidated_df.at[index, 'new_uom'] = 'lb'
        
    if 'gr' in item:
        consolidated_df.at[index, 'new_uom'] = 'g'

    if 'g' in item:
        consolidated_df.at[index, 'new_uom'] = 'g'
    
    if 'kg' in item:
        consolidated_df.at[index, 'new_uom'] = 'kg'
            
    
    numeric_vals = re.findall(r'\d+', item_descrip)
    if numeric_vals:
        numeric_val = int(numeric_vals[-1])  # Extract the last numeric value so for example for milk 2% it will not be 2 for numeric_val
        consolidated_df.at[index, 'total_weight'] = numeric_val * abs(row['qty'])


In [22]:
# if item has units of oz and it is a solid food item convert units to kg

liquid_foods = [44, 45, 46, 47, 48, 49, 50, 51, 55, 57, 58]
for index, row in consolidated_df.iterrows():
    item_descrip = row['item_descrip']
    item = re.findall(r'[a-zA-Z]+[\w%]*', item_descrip)
    
    if 'oz' in item and row['Category ID'] not in liquid_foods:
        consolidated_df.at[index, 'new_uom'] = 'kg'
        
        numeric_vals = re.findall(r'\d+', item_descrip)
        if numeric_vals:
            numeric_val = int(numeric_vals[-1]) # Extract the last numeric value so for example for milk 2% it will not be 2 for numeric_val
            consolidated_df.at[index, 'total_weight'] = numeric_val * abs(row['qty']) * 0.0283495 # conversion from oz to kg
        
        

In [23]:
# if item has units of oz and it is a liquid food item convert units to L

liquid_foods = [44, 45, 46, 47, 48, 49, 50, 51, 55, 57, 58]
for index, row in consolidated_df.iterrows():
    item_descrip = row['item_descrip']
    item = re.findall(r'[a-zA-Z]+[\w%]*', item_descrip)
    
    if 'oz' in item and row['Category ID'] in liquid_foods:
        consolidated_df.at[index, 'new_uom'] = 'L'
        
        numeric_vals = re.findall(r'\d+', item_descrip)
        if numeric_vals:
            numeric_val = int(numeric_vals[-1]) # Extract the last numeric value so for example for milk 2% it will not be 2 for numeric_val
            consolidated_df.at[index, 'total_weight'] = numeric_val * abs(row['qty']) * 0.0295735 # conversion from oz to L
        

In [24]:
consolidated_df.iloc[4728:4740, :]

,invoice_counter,invoice_num,invoice_total,item_num,qty,line_amount,adj_line_amount,uom,unit_cost,item_descrip,...,case_qty,purchase_uom,cat_num,cat_descrip,has_credit,credit_pak_amount,Food Category,Category ID,new_uom,total_weight
8956,248163,264425,33.80,3992,20.0,33.800,33.800,kg,1.6900,chick backs & necks -fresh,...,1.0,Kg,2,Food,Y,0,"poultry (chicken, turkey)",4.0,None,None
8959,249062,265060,672.95,3992,20.0,33.800,33.800,kg,1.6900,chick backs & necks -fresh,...,1.0,Kg,2,Food,Y,0,"poultry (chicken, turkey)",4.0,None,None
8961,250605,265503,563.29,3992,20.0,33.800,33.800,kg,1.6900,chick backs & necks -fresh,...,1.0,Kg,2,Food,Y,0,"poultry (chicken, turkey)",4.0,None,None
8963,250692,265725,645.44,3992,20.0,33.800,33.800,kg,1.6900,chick backs & necks -fresh,...,1.0,Kg,2,Food,Y,0,"poultry (chicken, turkey)",4.0,None,None
8960,249494,265315,194.57,3992,24.3,41.067,41.067,kg,1.6900,chick backs & necks -fresh,...,1.0,Kg,2,Food,Y,0,"poultry (chicken, turkey)",4.0,None,None
8958,248370,264802,384.37,3992,26.4,44.616,44.616,kg,1.6900,chick backs & necks -fresh,...,1.0,Kg,2,Food,Y,0,"poultry (chicken, turkey)",4.0,None,None
8957,248475,264629,3335.29,3992,52.0,87.890,87.890,kg,1.6902,chick backs & necks -fresh,...,1.0,Kg,2,Food,Y,0,"poultry (chicken, turkey)",4.0,None,None
8978,248250,264548,101.51,3994,4.4,61.556,61.556,kg,13.9900,chick breast bl wjo halal,...,1.0,Kg,2,Food,N,0,"poultry (chicken, turkey)",4.0,None,None
8982,251718,265877,332.87,3994,4.6,64.354,64.354,kg,13.9900,chick breast bl wjo halal,...,1.0,Kg,2,Food,N,0,"poultry (chicken, turkey)",4.0,None,None
8983,252416,266762,334.89,3994,7.7,107.723,107.723,kg,13.9900,chick breast bl wjo halal,...,1.0,Kg,2,Food,N,0,"poultry (chicken, turkey)",4.0,None,None


In [25]:
consolidated_df.iloc[2990:2995, :]

,invoice_counter,invoice_num,invoice_total,item_num,qty,line_amount,adj_line_amount,uom,unit_cost,item_descrip,...,case_qty,purchase_uom,cat_num,cat_descrip,has_credit,credit_pak_amount,Food Category,Category ID,new_uom,total_weight
7895,252525,9001459655,970.52,2697,1.0,59.26,59.26,cs,59.26,sauce pesto no nut,...,6.0,cs,2,Food,Y,0,tree nuts and seeds,26.0,None,None
7897,252640,9001524888,769.84,2697,1.0,59.26,59.26,cs,59.26,sauce pesto no nut,...,6.0,cs,2,Food,Y,0,tree nuts and seeds,26.0,None,None
7882,248855,9001123356,11011.81,2697,2.0,115.44,115.44,cs,57.72,sauce pesto no nut,...,6.0,cs,2,Food,N,0,tree nuts and seeds,26.0,None,None
7884,248857,9001173820,10187.78,2697,2.0,115.44,115.44,cs,57.72,sauce pesto no nut,...,6.0,cs,2,Food,N,0,tree nuts and seeds,26.0,None,None
7887,249798,9001250222,7684.21,2697,2.0,115.44,115.44,cs,57.72,sauce pesto no nut,...,6.0,cs,2,Food,N,0,tree nuts and seeds,26.0,None,None


In [26]:
consolidated_df['uom'].unique()

array(['cs', 'CA', 'CT', '6 pk', 'ea', 'LOAF', 'TUB', 'bottle', '750ML',
       '500ML', 'each', 'BIB 10L', '341ML', '473ML', 'pak', 'JUG', 'pail',
       'kg', '4L', 'bag', 'un', 'BX', 'box', 'PL', 'LG CAN', 'BG', 'can',
       'jar', 'lb', 'BASKET', 'BUNCH', 'PAK 3', '20UN', 'SHEET', 'L',
       'LAYER', '5KG', '355ML', '10LT', 'DOZ', 'ROLL', 'HEAD'],
      dtype=object)

In [27]:
pd.DataFrame(consolidated_df.iloc[1821:1845, :])

,invoice_counter,invoice_num,invoice_total,item_num,qty,line_amount,adj_line_amount,uom,unit_cost,item_descrip,...,case_qty,purchase_uom,cat_num,cat_descrip,has_credit,credit_pak_amount,Food Category,Category ID,new_uom,total_weight
3823,251531,9001425820,8341.31,2141,2.0,97.84,97.84,cs,48.92,cheese parm shred*,...,2.0,cs,2,Food,Y,0,cheese,6.0,None,None
3825,251532,9001437044,3843.55,2141,2.0,97.84,97.84,cs,48.92,cheese parm shred*,...,2.0,cs,2,Food,Y,0,cheese,6.0,None,None
3826,251645,9001447443,3982.36,2141,2.0,97.84,97.84,cs,48.92,cheese parm shred*,...,2.0,cs,2,Food,Y,0,cheese,6.0,None,None
3827,252008,9001459652,1853.19,2141,2.0,97.84,97.84,cs,48.92,cheese parm shred*,...,2.0,cs,2,Food,Y,0,cheese,6.0,None,None
3831,252009,9001470438,8409.18,2141,2.0,97.84,97.84,cs,48.92,cheese parm shred*,...,2.0,cs,2,Food,Y,0,cheese,6.0,None,None
3833,252508,9001489949,6474.60,2141,2.0,97.84,97.84,cs,48.92,cheese parm shred*,...,2.0,cs,2,Food,Y,0,cheese,6.0,None,None
3835,252201,9001501980,5725.83,2141,2.0,97.84,97.84,cs,48.92,cheese parm shred*,...,2.0,cs,2,Food,Y,0,cheese,6.0,None,None
3836,252298,9001501989,4740.23,2141,2.0,97.84,97.84,cs,48.92,cheese parm shred*,...,2.0,cs,2,Food,Y,0,cheese,6.0,None,None
3760,247877,9001094408,7899.82,2141,3.0,138.30,138.30,cs,46.10,cheese parm shred*,...,2.0,cs,2,Food,N,0,cheese,6.0,None,None
3785,249646,9001216334,8903.72,2141,3.0,138.30,138.30,cs,46.10,cheese parm shred*,...,2.0,cs,2,Food,N,0,cheese,6.0,None,None


In [28]:
consolidated_df['case_uom'].unique()

array(['SHEET', 'CT', 'DOZ', 'each', 'CAKE', 'ea', 'g', 'pak', 'LOAF',
       'lb', 'bottle', '750ML', '500ML', 'BIB 20L', '355ML', 'BIB 10L',
       'cs', '591ML', '341ML', 'can', '473ML', '710ml', 'JUG', 'L',
       'LG CAN', 'pail', 'Kg', '4L', 'bag', 'PTN', 'POUCH', 'un', '2L',
       'box', 'PIE', 'BLOCK', 'SM CAN', 'jar', '6L', 'BAR', 'BASKET',
       'HEAD', 'BUNCH', '5#', 'oz', 'PAK 3', 'SLEEVE', '100UN', '20UN',
       '500UN', 'TUB', '340ML', 'fl oz', '4KG', '414ML', '10LT', 'ROLL'],
      dtype=object)

In [29]:
for index, row in consolidated_df.iterrows():            
    if row['uom'] == 'kg' or row['case_uom'] == 'Kg':
        consolidated_df.at[index, 'new_uom'] = 'kg'
        consolidated_df.at[index, 'total_weight'] = abs(row['qty'])
        
    elif row['uom'] == 'lb' or row['case_uom'] == 'lb':
        consolidated_df.at[index, 'new_uom'] = 'lb'
        consolidated_df.at[index, 'total_weight'] = abs(row['qty'])
        
    elif row['uom'] == 'ml':
        consolidated_df.at[index, 'new_uom'] = 'ml'
        consolidated_df.at[index, 'total_weight'] = abs(row['qty'])
    
    elif row['uom'] == 'L' or row['case_uom'] == 'L':
        consolidated_df.at[index, 'new_uom'] = 'L'
        consolidated_df.at[index, 'total_weight'] = abs(row['qty'])
    
    elif row['uom'] == '2L' or row['case_uom'] == '2L':
        consolidated_df.at[index, 'new_uom'] = 'L'
        consolidated_df.at[index, 'total_weight'] = 2 * abs(row['qty'])
        
    elif row['uom'] == '6L' or row['case_uom'] == '6L':
        consolidated_df.at[index, 'new_uom'] = 'L'
        consolidated_df.at[index, 'total_weight'] = 6 * abs(row['qty'])
    
    elif row['case_uom'] == '10LT':
        consolidated_df.at[index, 'new_uom'] = 'L'
        consolidated_df.at[index, 'total_weight'] = 10 * abs(row['qty'])
        
    elif row['uom'] == 'g' or row['case_uom'] == 'g':
        consolidated_df.at[index, 'new_uom'] = 'g'
        consolidated_df.at[index, 'total_weight'] = abs(row['qty'])
    
    elif row['uom'] == '5KG':
        consolidated_df.at[index, 'new_uom'] = 'kg'
        consolidated_df.at[index, 'total_weight'] = 5 * abs(row['qty'])
        
    elif row['case_uom'] == '4KG':
        consolidated_df.at[index, 'new_uom'] = 'kg'
        consolidated_df.at[index, 'total_weight'] = 4 * abs(row['qty'])
  
    elif row['uom'] == '500ML' or row['case_uom'] == '500ML':
        consolidated_df.at[index, 'new_uom'] = 'ml'
        consolidated_df.at[index, 'total_weight'] = 500 * abs(row['qty'])
        
    elif row['uom'] == 'BIB 10L' or row['case_uom'] == 'BIB 10L':
        consolidated_df.at[index, 'new_uom'] = 'L'
        consolidated_df.at[index, 'total_weight'] = 10 * abs(row['qty'])
 
    elif row['case_uom'] == 'BIB 20L':
        consolidated_df.at[index, 'new_uom'] = 'L'
        consolidated_df.at[index, 'total_weight'] = 20 * abs(row['qty'])
        
    elif row['case_uom'] == '340ML':
        consolidated_df.at[index, 'new_uom'] = 'ml'
        consolidated_df.at[index, 'total_weight'] = 340 * abs(row['qty'])

    elif row['uom'] == '341ML' or row['case_uom'] == '341ML':
        consolidated_df.at[index, 'new_uom'] = 'ml'
        consolidated_df.at[index, 'total_weight'] = 341 * abs(row['qty'])
        
    elif row['case_uom'] == '414ML':
        consolidated_df.at[index, 'new_uom'] = 'ml'
        consolidated_df.at[index, 'total_weight'] = 414 * abs(row['qty'])
        
    elif row['uom'] == '355ML' or row['case_uom'] == '355ML':
        consolidated_df.at[index, 'new_uom'] = 'ml'
        consolidated_df.at[index, 'total_weight'] = 355 * abs(row['qty'])
    
    elif row['uom'] == '473ML' or row['case_uom'] == '473ML':
        consolidated_df.at[index, 'new_uom'] = 'ml'
        consolidated_df.at[index, 'total_weight'] = 473 * abs(row['qty'])
        
    elif row['case_uom'] == '591ML':
        consolidated_df.at[index, 'new_uom'] = 'ml'
        consolidated_df.at[index, 'total_weight'] = 591 * abs(row['qty'])
        
    elif row['case_uom'] == '710ml':
        consolidated_df.at[index, 'new_uom'] = 'ml'
        consolidated_df.at[index, 'total_weight'] = 710 * abs(row['qty'])
        
    elif row['uom'] == '750ML' or row['case_uom'] == '750ML':
        consolidated_df.at[index, 'new_uom'] = 'ml'
        consolidated_df.at[index, 'total_weight'] = 750 * abs(row['qty'])

    elif row['uom'] == '4L' or row['case_uom'] == '4L':
        consolidated_df.at[index, 'new_uom'] = 'L'
        consolidated_df.at[index, 'total_weight'] = 4 * abs(row['qty'])
    
    elif row['uom'] == '10LT':
        consolidated_df.at[index, 'new_uom'] = 'L'
        consolidated_df.at[index, 'total_weight'] = 10 * abs(row['qty'])

#     elif row['uom'] == 'LG CAN':
#         consolidated_df.at[index, 'new_uom'] = 'L'
#         consolidated_df.at[index, 'total_weight'] = abs(row['qty'])
       

In [30]:
pd.DataFrame(consolidated_df.iloc[1845:1875, :])

,invoice_counter,invoice_num,invoice_total,item_num,qty,line_amount,adj_line_amount,uom,unit_cost,item_descrip,...,case_qty,purchase_uom,cat_num,cat_descrip,has_credit,credit_pak_amount,Food Category,Category ID,new_uom,total_weight
3822,251398,9001408453,7231.50,2141,4.0,195.68,195.68,cs,48.92,cheese parm shred*,...,2.0,cs,2,Food,Y,0,cheese,6.0,kg,4.0
3832,252199,9001470485,5788.86,2141,4.0,195.68,195.68,cs,48.92,cheese parm shred*,...,2.0,cs,2,Food,Y,0,cheese,6.0,kg,4.0
3837,252509,9001501995,6688.74,2141,4.0,195.68,195.68,cs,48.92,cheese parm shred*,...,2.0,cs,2,Food,Y,0,cheese,6.0,kg,4.0
3839,252202,9001511810,3913.58,2141,4.0,195.68,195.68,cs,48.92,cheese parm shred*,...,2.0,cs,2,Food,Y,0,cheese,6.0,kg,4.0
3773,248666,9001155888,8612.47,2141,5.0,230.50,230.50,cs,46.10,cheese parm shred*,...,2.0,cs,2,Food,N,0,cheese,6.0,kg,5.0
3809,251527,9001364421,12483.64,2141,5.0,244.60,244.60,cs,48.92,cheese parm shred*,...,2.0,cs,2,Food,Y,0,cheese,6.0,kg,5.0
3758,247874,9001070944,8370.49,2141,6.0,276.60,276.60,cs,46.10,cheese parm shred*,...,2.0,cs,2,Food,N,0,cheese,6.0,kg,6.0
3772,248856,9001156740,8299.81,2141,6.0,276.60,276.60,cs,46.10,cheese parm shred*,...,2.0,cs,2,Food,N,0,cheese,6.0,kg,6.0
3779,248858,9001185346,5707.49,2141,6.0,276.60,276.60,cs,46.10,cheese parm shred*,...,2.0,cs,2,Food,N,0,cheese,6.0,kg,6.0
3787,249660,9001216528,9277.92,2141,6.0,276.60,276.60,cs,46.10,cheese parm shred*,...,2.0,cs,2,Food,N,0,cheese,6.0,kg,6.0


In [31]:
consolidated_df.shape

(18386, 33)

In [32]:
consolidated_df.isna().sum()

invoice_counter          0
invoice_num              0
invoice_total            0
item_num                 0
qty                      0
line_amount              0
adj_line_amount          0
uom                      0
unit_cost                0
item_descrip             0
invoice_date             0
supplier_name            0
store_num                0
type_bill_credit         0
cost_over_ride           0
line_amount_qty          0
case_uom                 0
supplier_num             0
prod_group_num           0
prod_group_descrip       0
order_code             261
compute_0022             0
pak_qty                  0
case_qty                 0
purchase_uom             0
cat_num                  0
cat_descrip              0
has_credit               0
credit_pak_amount        0
Food Category          687
Category ID            687
new_uom               9714
total_weight          8039
dtype: int64

In [33]:
consolidated_df[consolidated_df['new_uom'] == 'oz']

,invoice_counter,invoice_num,invoice_total,item_num,qty,line_amount,adj_line_amount,uom,unit_cost,item_descrip,...,case_qty,purchase_uom,cat_num,cat_descrip,has_credit,credit_pak_amount,Food Category,Category ID,new_uom,total_weight


In [34]:
total_rows = len(consolidated_df)
none_count = len(consolidated_df[consolidated_df['new_uom'].isnull()])
percentage = (none_count / total_rows) * 100

print(f"Percentage of rows with 'new_uom' equal to 'None': {percentage:.2f}%")


Percentage of rows with 'new_uom' equal to 'None': 52.83%


In [35]:
# The rows that have to be manually adjusted, do not have standard unit of measurement in uom column or in item_descrip

rows_with_none_uom = consolidated_df[consolidated_df['new_uom'].isnull()]
unique_item_nums = rows_with_none_uom['item_num'].unique()

result = consolidated_df[consolidated_df['item_num'].isin(unique_item_nums)]

result

,invoice_counter,invoice_num,invoice_total,item_num,qty,line_amount,adj_line_amount,uom,unit_cost,item_descrip,...,case_qty,purchase_uom,cat_num,cat_descrip,has_credit,credit_pak_amount,Food Category,Category ID,new_uom,total_weight
287,247973,9001094224,8190.64,1016,1.0,69.53,69.53,cs,69.53,brownie rocky road 12x16in tfc,...,2.0,cs,2,Food,N,0,"wheat/rye (bread, pasta, baked goods)",24.0,None,16.0
289,248381,9001112003,5367.66,1016,1.0,69.53,69.53,cs,69.53,brownie rocky road 12x16in tfc,...,2.0,cs,2,Food,N,0,"wheat/rye (bread, pasta, baked goods)",24.0,None,16.0
290,248383,9001132129,4406.91,1016,1.0,69.53,69.53,cs,69.53,brownie rocky road 12x16in tfc,...,2.0,cs,2,Food,N,0,"wheat/rye (bread, pasta, baked goods)",24.0,None,16.0
292,248663,9001144510,9116.11,1016,1.0,69.53,69.53,cs,69.53,brownie rocky road 12x16in tfc,...,2.0,cs,2,Food,N,0,"wheat/rye (bread, pasta, baked goods)",24.0,None,16.0
293,248997,9001185341,7931.74,1016,1.0,69.53,69.53,cs,69.53,brownie rocky road 12x16in tfc,...,2.0,cs,2,Food,N,0,"wheat/rye (bread, pasta, baked goods)",24.0,None,16.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15239,251896,00023332,547.24,70754,1.0,18.00,18.00,ea,18.00,ck bs sponge seasonal 1sheet,...,1.0,ea,2,Food,N,0,kitchen supplies,61.0,None,1.0
15240,253396,00023405,429.52,70754,1.0,18.00,18.00,ea,18.00,ck bs sponge seasonal 1sheet,...,1.0,ea,2,Food,N,0,kitchen supplies,61.0,None,1.0
15241,253395,00023427,504.52,70754,1.0,18.00,18.00,ea,18.00,ck bs sponge seasonal 1sheet,...,1.0,ea,2,Food,N,0,kitchen supplies,61.0,None,1.0
8603,251629,0000869104,968.35,70806,4.0,192.00,192.00,TUB,48.00,ice cream espresso 11.4l,...,1.0,TUB,2,Food,N,0,ice cream,7.0,None,16.0


In [36]:
# # Convert case_uom values to lowercase
# unique_case_uom = consolidated_df['case_uom'].str.lower().unique()

# # Calculate % of non-standard case units
# standard_units = ['case', 'box', 'pack']
# non_standard_count = np.sum([case_uom not in standard_units for case_uom in unique_case_uom])
# percentage_non_standard = (non_standard_count / len(unique_case_uom)) * 100

# print(f"Percentage of items needing manual adjustments to case units: {percentage_non_standard:.2f}%")


In [37]:
# # Read Coversions_List.csv that was created from 1_data preprocessing
# Conversions = pd.read_csv("C:/Users/smvan/CFFS-S23/CFFS-22-23/data/preprocessed/Conversions_List.csv")
# Conversions.dtypes

In [38]:
# Standard unit converter:
# Construct a unit converter for specific ingredients

# After this line below, spc_cov contains only the non-empty values from the 'ConversionId' column of the Conversions DataFrame.


# spc_cov = list(filter(None, Conversions['ConversionId'].tolist()))


# # Comments for spc_converter:
# # The function checks if ingredient is in the liquid_unit or solid_unit lists. If so, it calls std_converter(qty, uom) to 
# # convert the quantity and UOM to a standardized unit.

# # If uom is not in liquid_unit or solid_unit it checks if ingre is in spc_cov, if it is and the ConvertToUom is equal to grams 
# # then the function applies the factor to the qty argument to convert it to the standardized unit, and returns the result as
# # a tuple containing the converted quantity and uom. If no conversion found, then it calls std_converter(qty, uom)

# # If uom not in liquid_unit or solid_unit and if ingre is not in spc_cov then the function calls std_converter(qty, uom)

# def spc_converter(ingre, qty, uom):
#     if uom in liquid_unit + solid_unit:
#         return std_converter(qty, uom)
#     elif ingre in spc_cov:
#         conversion = Conversions.loc[(Conversions['ConversionId'] == ingre) & (Conversions['ConvertFromUom'] == uom)
#                                     & (Conversions['ConvertToUom'] == 'g')]
#         multiplier = conversion['Multiplier']
#         if multiplier.empty:
#             return std_converter(qty, uom)
#         else: 
#             Qty = float(qty)/float(multiplier)
#             Uom = conversion['ConvertToUom'].values[0]
#             return (Qty, Uom)
#     else:
#         return std_converter(qty, uom)

In [39]:
# # Display first 5 rows
# Conversions.head()

In [40]:
# # Add the specific conversion info from the newly-processed data to a unit conversion database
# Update_Conv = pd.read_csv(os.path.join("C:/Users/smvan/CFFS-S23/CFFS-22-23/data/cleaning/update/Conv_UpdateConv.csv"))
# Update_Conv

In [41]:
# # return dataframe with null values
# # There are no null values
# subset_conv = Update_Conv[Update_Conv["Multiplier"].isna()]
# subset_conv

In [42]:
# # takes a data frame and assigns a new column called Multiplier to the dataframe
# # uses iterrows() to iterate through the rows.Then subset_conv will have a new column named Multiplier with the computed values.
# def assign_multiplier(df):
#     for ind, row in df.iterrows():
#         df.loc[ind, "Multiplier"] = row["ConvertFromQty"] / row["ConvertToQty"]
        
# assign_multiplier(subset_conv)

# # May 8th change: We want to 
# assign_multiplier(Update_Conv)
# #subset_conv

In [43]:
# # Display the first 339 rows and all the columns
# Update_Conv = Update_Conv.iloc[0:339, :]
# Update_Conv

In [44]:
# # convert dataframe to csv file
# Update_Conv = pd.concat([Update_Conv, subset_conv], axis=0)
# Update_Conv.to_csv("C:/Users/smvan/CFFS-S23/CFFS-22-23/data/cleaning/update/Conv_UpdateConv.csv", index=False)

In [45]:
# # retrieves the value in the 'ConversionId' column using Update_Conv.loc[index, 'ConversionId'] 
# # and assigns it to the variable Id. 

# # row: a Series (1-d array in pandas) You can access the individual values of the row using the column headers as keys to the
# # like row['column_name'].

# # if the IDs of Update_Conv are the same IDs as Conversions then we drop it from the Conversions dataframe

# for index, row in Update_Conv.iterrows():
#     Id = Update_Conv.loc[index, 'ConversionId']
#     Conversions.drop(Conversions[Conversions['ConversionId'] == Id].index, inplace = True)

In [46]:
# # combine two data frames, reset the index and remove any duplicates in the concatenated data frame, 
# # and return the resulting data frame.

# # Added comments below on May 8th

# # frames variable is created as a list of two DataFrames, Conversions and Update_Conv.
# # pd.concat() function is used to concatenate the two DataFrames vertically (i.e., stack one on top of the other). 

# # drop_duplicates() method is used to remove any duplicate rows from the concatenated DataFrame based on all columns. 
# # The resulting DataFrame has only unique rows.

# # The resulting DataFrame is assigned back to the variable Conversions,

# # drop = True ensures that the old index is not added as a new column to the dataframe 
# # inplace=False parameter ensures that a new DataFrame is returned rather than modifying 
# # the original Conversions DataFrame in place.

# frames = [Conversions, Update_Conv]
# Conversions = pd.concat(frames).reset_index(drop=True, inplace=False).drop_duplicates()

In [47]:
# # Convert the DataFrame: Conversions into a csv file called Conversions_Added.csv
# path = os.path.join("C:/Users/smvan/CFFS-S23/CFFS-22-23/data/cleaning/Conversions_Added.csv")
# Conversions.to_csv(path, index = False, header = True)

In [48]:
# # Import standard unit conversion information and construct a dataframe
# Std_Unit = pd.read_csv("C:/Users/smvan/CFFS-S23/CFFS-22-23/data/external/standard_conversions.csv")
# Std_Unit.head()

In [49]:
# # Seperate uoms that converted to 'ml' or 'g'
# # Below we create 2 lists. 
# # list_unit contains list of unit of measurements that are being converted to milliliters 
# # solid_unit contains a list of unit of measurements that are being converted to grams
# # tolist() converts a Pandas Series or an array to a python list. 

# liquid_unit = Std_Unit.loc[Std_Unit['ConvertToUom'] == 'ml', 'ConvertFromUom'].tolist()
# solid_unit = Std_Unit.loc[Std_Unit['ConvertToUom'] == 'g', 'ConvertFromUom'].tolist()

In [50]:
# # Construct a standard unit converter
# def std_converter(qty, uom):
#     if uom in Std_Unit['ConvertFromUom'].tolist():
#         multiplier = Std_Unit.loc[Std_Unit['ConvertFromUom'] == uom, 'Multiplier']
#         Qty = float(qty)*float(multiplier)
#         Uom = Std_Unit.loc[Std_Unit['ConvertFromUom'] == uom, 'ConvertToUom'].values[0]
#     else:
#         Qty = qty
#         Uom = uom
#     return (Qty, Uom)

In [51]:
# Convert pounds to grams
# std_converter(0.25,'lb')

In [52]:
# # Construct a unit converter for specific ingredients

# # After this line below, spc_cov contains only the non-empty values from the 'ConversionId' column of the Conversions DataFrame.
# spc_cov = list(filter(None, Conversions['ConversionId'].tolist()))


# # Comments for spc_converter:
# # The function checks if ingredient is in the liquid_unit or solid_unit lists. If so, it calls std_converter(qty, uom) to 
# # convert the quantity and UOM to a standardized unit.

# # If uom is not in liquid_unit or solid_unit it checks if ingre is in spc_cov, if it is and the ConvertToUom is equal to grams 
# # then the function applies the factor to the qty argument to convert it to the standardized unit, and returns the result as
# # a tuple containing the converted quantity and uom. If no conversion found, then it calls std_converter(qty, uom)

# # If uom not in liquid_unit or solid_unit and if ingre is not in spc_cov then the function calls std_converter(qty, uom)

# def spc_converter(ingre, qty, uom):
#     if uom in liquid_unit + solid_unit:
#         return std_converter(qty, uom)
#     elif ingre in spc_cov:
#         conversion = Conversions.loc[(Conversions['ConversionId'] == ingre) & (Conversions['ConvertFromUom'] == uom)
#                                     & (Conversions['ConvertToUom'] == 'g')]
#         multiplier = conversion['Multiplier']
#         if multiplier.empty:
#             return std_converter(qty, uom)
#         else: 
#             Qty = float(qty)/float(multiplier)
#             Uom = conversion['ConvertToUom'].values[0]
#             return (Qty, Uom)
#     else:
#         return std_converter(qty, uom)

In [53]:
# import math
# spc_cov2 = [item for item in spc_cov if not(pd.isnull(item)) == True]
# spc_cov2

**Below I have extracted all unique items with Nonstandard Unit of Measurement in the consolidated report that need to be manually adjusted**

In [54]:
# # Filter out the items whose unit information is unknown 

# # We find the column names
# col_names = list(consolidated_df.columns.values)

# # Create a Items_Nonstd list
# Items_Nonstd = []

# # If the unit of measurement is not grams or ml and ingredient id starts with I and the ingredient is not in ConversionId column of Conversions 
# # then we add it to Items_Nonstd list
# for index, row in consolidated_df.iterrows():
#     Uom = consolidated_df.loc[index,'uom']
#     if Uom not in ['kg', 'lbs', 'oz', 'g', 'ml'] and Uom not in liquid_unit + solid_unit:
#         Dict = {}
#         Dict.update(dict(row))
#         Items_Nonstd.append(Dict)

# # Create a DataFrame from Items_Nonstd list
# Items_Nonstd = pd.DataFrame(Items_Nonstd, columns = col_names)
# # Remove duplicate ingredients of the same properties so that Items_Nonstd has only unique rows. 
# Items_Nonstd.drop_duplicates(subset=['item_num'], inplace=True,)
# Items_Nonstd